# Statistics and Scientific Computing Examples

Created: March 2024

By: Kerrie Geil, Associate Research Professor, Geosystems Research Institute, Mississippi State University

----

You will need to have the following packages in your conda environment to run this notebook: 

numpy, xarray, pandas, scipy, matplotlib, cartopy, scikit-learn, netcdf4, ipykernel

Most of these packages should already be in your dataprep environment but you will probably have to install cartopy and scikit-learn and possibly scipy.

Hint: to check the list of packages installed in your conda environment, first activate the environment at the command line, then command: conda list

**Important: open this notebook on a development node the first time you run it** because cartopy requires internet access to download coastlines for plotting.

Topics Covered
- Calculating anomalies
- Pearson r correlation (multiple packages)
- Time dimension rolling mean
- Fancy indexing
- Stacking and masking
- Focal operation package suggestions

In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import scipy.stats
import numpy.testing as npt

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
from sklearn.feature_selection import r_regression

In [ ]:
basedir='/path/to/our/shared/datasets/dir/'
outputdir=basedir+'temporary/stats_demo_outputs/'

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
    
year_start='1950'
year_end='2023'

# Read Nino 3.4 Index from a text file

1949-2023 (75 years, 12 months)

In [ ]:
# our data file contains rows with the year and 12 monthly anomaly values for the Nino 3.4 area 
# skip the first column of years and read in the other 12 columns of data (python indexing is not inclusive at the end)
# the base period for the anomalies is 1981-2010
# data=np.loadtxt(basedir+'temporary/climate_indices/NOAA-PSL_nino3.4.txt', usecols=(np.arange(1,13)))
nino_ind_raw=np.loadtxt(basedir+'temporary/climate_indices/NOAA-PSL_nino3.4_anomaly.txt', usecols=(np.arange(1,13)))

# collapse data to a 1D timeseries
nino_ind=nino_ind_raw.flatten()

# print some info
print(nino_ind_raw.shape) # see that the data is 2D
print(nino_ind.shape) # now it's 1D
print(nino_ind.min(),nino_ind.max()) # data value range
nino_ind[0:24] # first two years of values

In [ ]:
# make an array of datetimes to match the nino_ind data
time=pd.date_range('1949-01','2023-12',freq='MS')
time

In [ ]:
fig=plt.figure(figsize=(15,2))
plt.plot(time,nino_ind, linewidth=1)
plt.axhline(y=0., color='grey', linestyle='--', linewidth=0.5)
plt.title('Nino3.4 Index Anomaly')
plt.show()

# Read NCEP-NCAR-1 Reanalysis precipitation monthly mean

In [ ]:
# access all the data in the netcdf file as an xarray dataset
# note: for tif files you can use xr.open_dataset with engine='rasterio' or rioxarray rio.open_rasterio
ds=xr.open_dataset(basedir+'temporary/NCEP-NCAR-1/prate.sfc.mon.mean.nc')
ds

In [ ]:
# pull the precip data out as an xarray data array and subset in time
pr=ds.prate.sel(time=slice(year_start,year_end))

# adjust precip data
pr=pr*86400.               # kg/m2/s --> mm/day
pr.attrs['units']='mm/day' # update metadata 
pr=xr.where(pr<0.,0,pr)    # anywhere pr is negative fill with zero
pr

# Calculate anomalies in the precip data

using a base periodo 1981-2010

In [ ]:
# calculate the climatological values (30 year mean for each month)
clim= pr.sel(time=slice('1981','2010')).groupby('time.month').mean()
clim

In [ ]:
# calculate anomalies
pr_anom=pr.groupby('time.month') - clim
pr_anom

In [ ]:
# plot anomalies for a single month

fig=plt.figure(figsize=(10,4))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree()) # projection comes from cartopy package
ax.coastlines(linewidth=0.5)  # coastlines come from cartopy package      

cbar_kwargs = {'label':'pr anomaly (mm/day)'} # modifications to the colorbar

pr_anom.sel(time='1990-08-01').plot(ax=ax,cmap='bwr_r',cbar_kwargs=cbar_kwargs) # this is xarray plotting which is based on matplotlib package
plt.title('August 1990')
plt.show()

# Correlate precip anomalies with nino 3.4 index

#### pearson correlation of nino1D timeseries (1950-2023) with every pixel timeseries (1950-2023) of pr_anom


A number of packages have functions you can use to compute pearson correlation.

- **scipy:** [scipy.stats.pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), inputs can be xarray or numpy arrays but this function only operates on 1D inputs so you have to loop through data points which could be slow. This function also returns the one- or two-sided pvalue and confidence interval. I use this when I looking to get a quick estimation of the p values.

- **xarray:** [xr.corr](https://docs.xarray.dev/en/stable/generated/xarray.corr.html), you need both arrays to be xarray objects with a shared dimension and it outputs the Pearson correlation without any additional statistical information. It does operate on multiple dimensions so you can input two 3D arrays (time,lat,lon) and correlate over time to easily produce a point-to-point correlation map. I use this when I'm looking to do quick point-by-point correlations between multiple 3D data variables.

- **numpy:** [np.corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html), takes numpy arrays inputs that must be the same shape and it outputs the correlation matrix without any additional statistical information. Only operates on 1D and 2D arrays so if you have a array with (time,lat,lon) you need to collapse the space dimensions and then reshape back to 3D after the correlation. You also have to do extra work to pull the correlation values out of the matrix.

- **scikit learn:** [sklearn.feature_selection.r_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.r_regression.html#sklearn.feature_selection.r_regression), takes numpy or xarray arrays but only operates on either two 1D array inputs or one 1D and one 2D array inputs, so you need to collapse the space dimension and you can't correlate 3D variables easily. It returns the correlation coefficients without any additional statistical info.

- **custom written funcion:** write your own function for both pval (and r if you want) if you are interested in modifying to account for autocorrelation 

#### scipy.stats.pearsonr example

In [ ]:
y=nino_ind[12:]  # subset time to start at 1950, this is a numpy array
X=pr_anom.stack(space=('lat','lon'))  # reshape to 2D xarray ('time','space')
y.shape, X.shape

In [ ]:
results=[scipy.stats.pearsonr(X[:,ipt],y) for ipt in range(X.shape[1])]  # loop thru space points and return results in a list
len(results), results[0], results[0].statistic, results[0].confidence_interval(confidence_level=0.95)  # examples of how to access the info returned      

In [ ]:
# get the results back into arrays for plotting

r_list=[results[ipt].statistic for ipt in range(len(results))] # pull r into its own list
r=xr.DataArray(r_list,dims=['space'],coords={'space':X.space}).unstack() # create xarray object with lat and lon labels

# do the same for p
p_list=[results[ipt].pvalue for ipt in range(len(results))] # pull p into its own list
p=xr.DataArray(p_list,dims=['space'],coords={'space':X.space}).unstack() # create xarray object with lat and lon labels

r

In [ ]:
# plot correlation masked with pvalue

fig=plt.figure(figsize=(10,4))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree()) # projection comes from cartopy package
ax.coastlines(linewidth=0.5)  # coastlines come from cartopy package      

cbar_kwargs = {'label':'pearson r'} # modifications to the colorbar

siglvl=0.05
r.where(p<=siglvl).plot(ax=ax,cmap='RdBu',cbar_kwargs=cbar_kwargs) # this is xarray .where and .plot
plt.title('Correlation between monthly precip and nino3.4 anomalies, p<='+str(siglvl))
plt.show()

#### xr.corr example

In [ ]:
# with xarray

y_xr=xr.DataArray(y,dims='time',coords={'time':time[12:]}) # convert numpy array to xarray
r_xr=xr.corr(X,y_xr,dim='time').unstack()

# plot correlation
fig=plt.figure(figsize=(10,4))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree()) # projection comes from cartopy package
ax.coastlines(linewidth=0.5)  # coastlines come from cartopy package      

cbar_kwargs = {'label':'pearson r'} # modifications to the colorbar

r_xr.plot(ax=ax,cmap='RdBu',cbar_kwargs=cbar_kwargs) # this is xarray .where and .plot and colorbar properties have to be passed here with kwargs
plt.title('Correlation between monthly precip and nino3.4 anomalies')
plt.show()

#### sklearn.feature_selection.r_regression example

In [ ]:
r_sk=r_regression(X,y)  # scikit-learn function
r_sk=r_sk.reshape((len(pr_anom.lat),len(pr_anom.lon))) # reshape to 2D numpy array

# notice r_sk is not in an xarray data structure here
# so we can't use xarray plotting unless we convert r_sk to xarray
# we could easily convert to xarray object using the same method as for y_xr above
# I'm not doing that here so you can see a different way to plot

# you will notice xarray does some things automatically to make plots look nicer that don't occur by default below, for example
# - when using a 2-color cmap, xarray will automatically make the colorbar min and max extents the same so that the pos/neg color transition sits right at zero
# - xarray will also label certain things based on the variable's metadata/labels. this doesn't really apply to the above, but thought I'd mention it


# plot
fig=plt.figure(figsize=(10,4))
ax=fig.add_subplot(111,projection=ccrs.PlateCarree())  # projection comes from cartopy package
ax.coastlines(linewidth=0.5)  # coastlines come from cartopy package          
plt.pcolormesh(pr_anom.lon.data,pr_anom.lat.data,r_sk,cmap='RdBu')  # this is a matplotlib.pyplot function, whereas before we were using .plot from xarray
plt.title('correlation between monthly nino3.4 and precip anomalies 1950-2023')
plt.colorbar(label='pearson r') # colorbar properties are passed this way
plt.show()

In [ ]:
# manually set the colorbar limits so that zero falls at the color transition

# plot
fig=plt.figure(figsize=(10,4))
ax=fig.add_subplot(111,projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5)        
plt.pcolormesh(pr_anom.lon.data,pr_anom.lat.data,r_sk,cmap='RdBu',vmax=r_sk.max(),vmin=-r_sk.max())
plt.title('correlation between monthly nino3.4 and precip anomalies 1950-2023')
plt.colorbar(label='pearson r')
plt.show()

#### custom function example

In [ ]:
def pearson_r(timeseries,data_arr):
    ''' accepts a 1D xarray timeseries with dim label 'time'
        and a 2D or 3D xarray array with the same 'time' dim'''
    
    #1. Compute data length, mean and standard deviation along time axis: 
    n=timeseries.shape[0]
    xmean = timeseries.mean('time')
    ymean = data_arr.mean('time')
    xstd  = timeseries.std('time')
    ystd  = data_arr.std('time')
                  
    #2. Compute covariance along time axis
    cov   =  np.sum((timeseries - xmean)*(data_arr - ymean), axis=0)/n
    cov=cov.where(np.isfinite(ymean),np.nan) # np.sum sets the sum of all nan to 0, change these back to nan

    #3. Compute correlation along time axis
    cor   = cov/(xstd*ystd)
    
    # you could also just replace all of the above with one of the package functions we just covered
    
    #4. Compute p value
    dof=n-2  # if you want to account for autocorrelation here's where you'd do it
    tstats = cor*np.sqrt(dof)/np.sqrt(1-cor**2)  # students t
    pval   = scipy.stats.t.sf(abs(tstats), dof)*2 # two tailed using effective sample size for dof
    pval   = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)    
    
    class output:
        r=cor
        p=pval
        
    return output

In [ ]:
results=pearson_r(y_xr,pr_anom)

In [ ]:
# plot correlation
fig=plt.figure(figsize=(10,4))

ax=fig.add_subplot(111,projection=ccrs.PlateCarree()) # projection comes from cartopy package
ax.coastlines(linewidth=0.5)  # coastlines come from cartopy package      

cbar_kwargs = {'label':'pearson r'} # modifications to the colorbar

results.r.where(results.p<=siglvl).plot(ax=ax,cmap='RdBu',cbar_kwargs=cbar_kwargs) # this is xarray .where and .plot and colorbar properties have to be passed here with kwargs
plt.title('Correlation between monthly precip and nino3.4 anomalies, pval<='+str(siglvl))
plt.show()

In [ ]:
# let's test to see if r and p from our custom function and different packages are identical
# if identical, nothing prints

npt.assert_allclose(results.r,r,rtol=0,atol=1e-5,err_msg='r scipy NOT EQUAL!')
npt.assert_allclose(results.r,r_xr,rtol=0,atol=1e-5,err_msg='r_xr NOT EQUAL!')
npt.assert_allclose(results.r,r_sk,rtol=0,atol=1e-5,err_msg='r_sk NOT EQUAL!')

npt.assert_allclose(results.p,p,rtol=0,atol=1e-5,err_msg='p scipy NOT EQUAL!')

# Rolling in time example

### finding el nino / la nina events from nino3.4 index

In [ ]:
# first do 5 month rolling mean
nino_ind_xr=xr.DataArray(nino_ind,dims='time',coords={'time':time}) # numpy to xarray, using 1949 so we don't lose data with the rolling mean
nino_rollmean=nino_ind_xr.rolling(time=5,center=False).mean()
nino_rollmean

In [ ]:
# el nino events are where nino_rollmean > 0.4 for 6 consecutive months 
# la nina events are where nino_rollmean <-0.4 for 6 consecutive months

# first put the data in windows of length 6 months
nino_windows=nino_rollmean.rolling(time=6,center=False).construct('window')
nino_windows

# other time window and fancy indexing techniques

In [ ]:
# now create boolean arrays where the index window meets the threshold for nino/nina events
# the window is 6 months long, if all months above threshold, this collapses the window dimension and fills True in for the last month
nino=(nino_windows>0.4).all(dim='window')
nina=(nino_windows<-0.4).all(dim='window')
nino

In [ ]:
# to get all the nino/nina months we need to find each True and fill the 5 preceeding months with True also
# this will give us the full length of each nino/nina event

nino_window_true=np.argwhere(nino.data).squeeze()  # indexes of where nino is True
nina_window_true=np.argwhere(nina.data).squeeze()  # indexes of where nino is True
print(nino_window_true.shape, nino_window_true[0:5])

# add indexes of the preceeding 5 months
nino_list=list(nino_window_true) # numpy array to list
nina_list=list(nina_window_true) # numpy array to list
for i in range(1,6):
    nino_list.extend(list(nino_window_true-i)) # add additional indexes to the list
    nina_list.extend(list(nina_window_true-i)) # add additional indexes to the list
    
# since above could and does contain duplicates...
nino_indexes=np.unique(nino_list) # this removes duplicates and sorts ascending
nina_indexes=np.unique(nina_list) # this removes duplicates and sorts ascending
nino_indexes

In [ ]:
# fix up nino and nina with this new info

nino[nino_indexes]=True
nina[nina_indexes]=True

In [ ]:
# create a mask where nino event months = 1, nina event months = -1, and everything else = 0
events=nino_rollmean.copy()
events[:]=0  # set all values to 0

events=xr.where(nino,1,events)  # fill with 1 where nino boolean array is True
events=xr.where(nina,-1,events) # fill with -1 where nina boolean array is True
# events=events.sel(time=slice(year_start,year_end)) # subset in time


In [ ]:
# see how many of each value is in the events array
unique, counts = np.unique(events, return_counts=True)
for i in range(len(unique)):
    print(unique[i],counts[i])

In [ ]:
# get the start and stop of each event

nino_bounds=[]
istart=nino_indexes[0]

for i in range(nino_indexes.shape[0]-1):    
    if nino_indexes[i+1]==nino_indexes[i]+1:
        iend=nino_indexes[i+1]
    else:
        nino_bounds.append((istart,iend))
        istart=nino_indexes[i+1]
        
        
nina_bounds=[]
istart=nina_indexes[0]

for i in range(nina_indexes.shape[0]-1):    
    if nina_indexes[i+1]==nina_indexes[i]+1:
        iend=nina_indexes[i+1]
    else:
        nina_bounds.append((istart,iend))
        istart=nina_indexes[i+1]        
    
nino_bounds   

In [ ]:
# put tons of stuff on a plot

fig=plt.figure(figsize=(20,4))
plt.plot(time,nino_ind_xr, marker=10, markersize=3,linestyle='None') # monthly nino index as triangles
plt.plot(time,nino_rollmean,linewidth=1)                             # 5-month rolling mean nino index as a line
plt.axhline(y=0., color='grey', linestyle='--', linewidth=0.5)       # dashed grey line at zero
plt.axhline(y=-0.4, color='grey', linestyle='-', linewidth=0.5)      # solid grey line at nina threshold 
plt.axhline(y=0.4, color='grey', linestyle='-', linewidth=0.5)       # solid grey line at nino threshold

# blue shading during nino events
for ievent in range(len(nino_bounds)):
    time1=time[nino_bounds[ievent][0]]
    time2=time[nino_bounds[ievent][1]]
    plt.axvspan(time1,time2, color='cyan', alpha=0.25, lw=0)

# yellow shading during nina events    
for ievent in range(len(nina_bounds)):
    time1=time[nina_bounds[ievent][0]]
    time2=time[nina_bounds[ievent][1]]
    plt.axvspan(time1,time2, color='gold', alpha=0.25, lw=0)


# Stacking

Stacking usually means something slightly different in python than it does in the GIS world. Instead of meaning aligning data layers for computation, in python stacking usually means reducing dimensions. We saw this above in the correlation section when we changed our 3D array (time,lat,lon) to a 2D array of (time,space) with X=pr_anom.stack(space=('lat','lon')). In python, we put our data into multi-dimensional arrays where time is one dimension and lat, lon or space are dimensions as well. So, the "stack" terminology in the GIS/image processing world is really just the time dimension of our arrays. To calculate on multiple different 2- or 3-D arrays simultaneously we first usually regrid/resample to a common grid. Once our data is all in arrays on a common grid we can easily calculate things across the arrays. 

here our precip, tmin, tmax are all already on the same grid so we can skip resampling, everything has dimensions (time: 888, lat: 94, lon: 192)

# Masking

Masking is generally done with .where in python. 

There should be a .where available for whichever data structure that your data is in (e.g. xarray data array use xr.where, numpy array use np.where, pandas dataframe use pd.where, etc). 

[Numpy also has a module for constructing masked arrays](https://numpy.org/doc/stable/reference/maskedarray.generic.html). Personally, I never use this but am linking it here in case you want to look into it.

In [ ]:
# get more NCEP NCAR Reanalysis data
ds1=xr.open_dataset(basedir+'temporary/NCEP-NCAR-1/tmin.2m.mon.mean.nc').sel(time=slice(year_start,year_end))
tmin=ds1.tmin

ds2=xr.open_dataset(basedir+'temporary/NCEP-NCAR-1/tmax.2m.mon.mean.nc').sel(time=slice(year_start,year_end))
tmax=ds2.tmax

tmax

In [ ]:
# masking example
# at every month, find locations where 
# mean monthly precip is less than 10 mm/day and 
# mean monthly temperature range is greater than 20C

# masking with xarray, returns an xarray object, retains metadata labels
# each condition needs to be inside () like (condition)&(condition)&(condition)
mask_xr=xr.where((pr<10)&((tmax-tmin)>20),1,0)

# masking with numpy, returns a numpy object, losing all metadata labels
mask_np=np.where((pr<10)&((tmax-tmin)>20),1,0)

In [ ]:
# plot xarray result
fig=plt.figure(figsize=(20,4))
ax=fig.add_subplot(121,projection=ccrs.PlateCarree()) # 121 is rows,columns,plot_number
ax.coastlines(linewidth=0.5)        
mask_xr.sel(time='2020-08').plot(ax=ax) # this is xarray .plot 
plt.title('Aug 2020, mask_xr, where pr<10 & (tmax-tmin)>20')
            

# plot numpy result

# what index in time is Aug 2020?
t_ind=list(mask_xr.time.data).index(mask_xr.time.sel(time='2020-08').data)

ax=fig.add_subplot(122,projection=ccrs.PlateCarree()) # 122 is rows,columns,plot_number
ax.coastlines(linewidth=0.5)      
plt.pcolormesh(mask_xr.lon.data,mask_xr.lat.data,mask_np[t_ind,:,:])  # pcolormesh is from matplotlib.pyplot
plt.colorbar()
plt.title('Aug 2020, mask_np, where pr<10 & (tmax-tmin)>20')

plt.tight_layout()
plt.show()

In [ ]:
# apply the mask to a calculation (also done with .where)

# calculate something only at the locations where mask=1
# this could be any calculation, here we'll use the temperature range
masked_calc=(tmax-tmin).where(mask_xr)  # xarray objects, xarray .where, result returned as xarray object with labels

# plot result
fig=plt.figure(figsize=(20,4))
ax=fig.add_subplot(121,projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5)        
cbar_kwargs = {'label':'degrees C'} # modifications to the colorbar
masked_calc.sel(time='2020-08').plot(ax=ax,cbar_kwargs=cbar_kwargs)  
plt.title('Aug 2020, masked mean temperature range with xarray')


# numpy version of above
masked_calc_np=np.where(mask_np,(tmax.data-tmin.data),np.nan)  # numpy objects, numpy .where, result returned as numpy object (no labels)

# plot result
ax=fig.add_subplot(122,projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5)      
plt.pcolormesh(mask_xr.lon.data,mask_xr.lat.data,masked_calc_np[t_ind,:,:]) 
plt.colorbar(label='degrees C')
plt.title('Aug 2020, masked mean temperature range with numpy')

plt.tight_layout()
plt.show()

In [ ]:
# reproduce the above but with discreet colorbars
# this is easy to do with xarray plotting

# type the list of levels out manually
fig=plt.figure(figsize=(20,4))
ax=fig.add_subplot(121,projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5)        
cbar_kwargs = {'label':'degrees C'} # modifications to the colorbar
masked_calc.sel(time='2020-08').plot(ax=ax,levels=[20,22,24,26,28],cbar_kwargs=cbar_kwargs)  
plt.title('Aug 2020, masked mean temperature range')


# use range to create a list of levels
levels=range(21,29)

ax=fig.add_subplot(122,projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5)        
cbar_kwargs = {'label':'degrees C'} # modifications to the colorbar
masked_calc.sel(time='2020-08').plot(ax=ax,levels=levels,cbar_kwargs=cbar_kwargs)  
plt.title('Aug 2020, masked mean temperature range')

plt.tight_layout()
plt.show()

# focal operations

I'm not too familiar with this but I think in python, focal operations are generally offered in image processing packages.  Some packages that may be useful are:
- [scipy.ndimage](https://docs.scipy.org/doc/scipy/reference/ndimage.html) works on numpy arrays, I would start here
- [dask_image.ndfilters](https://image.dask.org/en/latest/dask_image.ndfilters.html) scipy.ndimage functions written for dask arrays (chunked big data)
- [focal_stats](https://focal-stats.readthedocs.io/en/latest/api.html) maybe the simplest, but limited functions. basically a numpy extension
- [xarray-spatial](https://pypi.org/project/xarray-spatial/) basically an xarray extension. This is new and it looks like access to the documentation may not be available yet. I've never used this but might be something to keep an eye on.
- [PIL/pillow](https://pillow.readthedocs.io/en/stable/) basic processing for data in image formats
- [scikit image](https://scikit-image.org/docs/stable/) image processing, lots of functions but maybe not great for working with a time dimension
- [opencv](https://docs.opencv.org/4.x/d2/d96/tutorial_py_table_of_contents_imgproc.html) probably similar to scikit image? I've never used it
